In [15]:
import sys
import subprocess

def ensure_package(pkg):
    try:
        __import__(pkg)
    except ImportError:
        print(f"Installing {pkg}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for package in ["pandas"]:
    ensure_package(package)

print("All required packages are installed.")

All required packages are installed.


In [16]:
from __future__ import annotations
import pandas as pd
import urllib.request
from pathlib import Path
import shutil
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')
print("Libraries imported successfully")

Libraries imported successfully


In [17]:
# Data source URLs organized by year and type
from pathlib import Path
CRASHES = {
    2021: "https://data.ca.gov/datastore/dump/d08692e2-6d36-487e-bca0-28cd127a626f?format=csv",
    2022: "https://data.ca.gov/datastore/dump/7828780b-117b-455e-9275-986ad3ffde50?format=csv",
    2023: "https://data.ca.gov/datastore/dump/436642c0-cd04-4a4c-b45e-564b66437476?format=csv",
    2024: "https://data.ca.gov/datastore/dump/f775df59-b89b-4f82-bd3d-8807fa3a22a0?format=csv",
    2025: "https://data.ca.gov/datastore/dump/9f4fc839-122d-4595-a146-43bc4ed16f46?format=csv",
}
PARTIES = {
    2021: "https://data.ca.gov/datastore/dump/754fe00c-f3bf-4f2f-80d0-ed4aa7b89b77?format=csv",
    2022: "https://data.ca.gov/datastore/dump/9ef51178-51cb-4939-9344-2d0907740580?format=csv",
    2023: "https://data.ca.gov/datastore/dump/84376be5-548b-44e3-8ebc-73e8a2ca9945?format=csv",
    2024: "https://data.ca.gov/datastore/dump/93892d36-017b-4a2a-bc0b-f1f385060b96?format=csv",
    2025: "https://data.ca.gov/datastore/dump/a2676918-a825-4b77-8e5c-6eadb38d6b1a?format=csv",
}
INJURED_WITNESSES = {
    2021: "https://data.ca.gov/datastore/dump/616a9850-27cb-4012-b6e7-90a2e495900a?format=csv",
    2022: "https://data.ca.gov/datastore/dump/2d9e8bef-d5a2-402e-82eb-6386ad4d09f7?format=csv",
    2023: "https://data.ca.gov/datastore/dump/1dfc36fa-a5dd-4616-b9b0-ff55699e299a?format=csv",
    2024: "https://data.ca.gov/datastore/dump/a36a0078-d7e1-4244-8337-0a59433c9b84?format=csv",
    2025: "https://data.ca.gov/datastore/dump/10184ea3-7411-42d8-87a6-17039b58f04b?format=csv",
}
YEARS = [2021, 2022, 2023, 2024, 2025]
# Setup directories - portable across systems
BASE_DIR = Path.cwd()
RAW_DIR = BASE_DIR / "data" / "raw"
OUTPUT_DIR = BASE_DIR / "analysis_outputs"
# Create directories if they don't exist
RAW_DIR.mkdir(parents=True, exist_ok=True)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
print(f"Configuration complete")
print(f"Base directory: {BASE_DIR}")
print(f"Raw data directory: {RAW_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

Configuration complete
Base directory: /workspaces/crash-data
Raw data directory: /workspaces/crash-data/data/raw
Output directory: /workspaces/crash-data/analysis_outputs


In [27]:
def download_if_missing(url: str, dest: Path) -> None:
    """Download file from URL if it doesn't exist locally or is empty."""
    if dest.exists() and dest.stat().st_size > 0:
        print(f"✓ exists: {dest.name}")
        return
    
    print(f"↓ downloading: {dest.name}")
    tmp = dest.with_suffix(dest.suffix + ".part")
    
    try:
        urllib.request.urlretrieve(url, tmp)
        tmp.replace(dest)
        print(f"✓ saved: {dest.name}")
    except Exception as e:
        print(f"✗ failed to download {dest.name}: {e}")
        if tmp.exists():
            tmp.unlink()
        raise
    finally:
        if tmp.exists():
            tmp.unlink(missing_ok=True)
def download_all_data() -> None:
    """Download all required data files."""
    print("Starting data download...")
    
    for year in YEARS:
        print(f"\n--- Year {year} ---")
        download_if_missing(CRASHES[year], RAW_DIR / f"crashes_{year}.csv")
        download_if_missing(PARTIES[year], RAW_DIR / f"parties_{year}.csv")
        download_if_missing(INJURED_WITNESSES[year], RAW_DIR / f"injured_witnesses_{year}.csv")
    
    print("\n All downloads complete")
# Run the download
download_all_data()

Starting data download...

--- Year 2021 ---
✓ exists: crashes_2021.csv
✓ exists: parties_2021.csv
✓ exists: injured_witnesses_2021.csv

--- Year 2022 ---
✓ exists: crashes_2022.csv
✓ exists: parties_2022.csv
✓ exists: injured_witnesses_2022.csv

--- Year 2023 ---
✓ exists: crashes_2023.csv
✓ exists: parties_2023.csv
✓ exists: injured_witnesses_2023.csv

--- Year 2024 ---
✓ exists: crashes_2024.csv
✓ exists: parties_2024.csv
✓ exists: injured_witnesses_2024.csv

--- Year 2025 ---
✓ exists: crashes_2025.csv
✓ exists: parties_2025.csv
✓ exists: injured_witnesses_2025.csv

 All downloads complete


In [28]:
def norm(column_name: str) -> str:
    """Normalize column names for consistent matching."""
    return "".join(ch for ch in str(column_name).strip().lower() if ch.isalnum())
def build_address(row: pd.Series) -> str:
    """Build a readable address from crash location components."""
    primary = str(row.get("PrimaryRoad", "")).strip()
    secondary = str(row.get("SecondaryRoad", "")).strip()
    sec_direction = str(row.get("SecondaryDirection", "")).strip()
    sec_distance = str(row.get("SecondaryDistance", "")).strip()
    if primary and secondary and secondary.lower() != "nan":
        return f"{primary} & {secondary}"
    if primary:
        if (
            sec_distance
            and sec_distance.lower() != "nan"
            and sec_direction
            and sec_direction.lower() != "nan"
        ):
            return f"{sec_direction} {sec_distance} from {primary}"
        return primary
    return ""
def load_crashes_data(year: int) -> pd.DataFrame:
    """Load and return crashes data for a given year."""
    crashes_path = RAW_DIR / f"crashes_{year}.csv"
    if not crashes_path.exists():
        raise FileNotFoundError(f"Crashes data for {year} not found: {crashes_path}")
    
    return pd.read_csv(crashes_path, low_memory=False)
def load_parties_data_chunked(year: int, chunk_size: int = 200_000):
    """Load parties data in chunks for memory efficiency."""
    parties_path = RAW_DIR / f"parties_{year}.csv"
    if not parties_path.exists():
        raise FileNotFoundError(f"Parties data for {year} not found: {parties_path}")
    
    return pd.read_csv(
        parties_path,
        chunksize=chunk_size,
        engine="python",
        on_bad_lines="skip",
    )
print("Utility functions defined")

Utility functions defined


In [34]:
def run_analysis() -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Run the complete SF crash analysis.
    
    Returns:
        Tuple of (age_frequency, nonblank_age_frequency, bicyclist_counts, bike_locations)
    """
    print("Starting SF Crash Analysis...")
    
    all_non_driver_ages = []
    yearly_bicyclist_counts = []
    bike_location_frames = []
    for year in YEARS:
        print(f"\n--- Processing {year} ---")
        
        # Load crashes data
        crashes = load_crashes_data(year)
        crash_cols = {norm(c): c for c in crashes.columns}
        # Extract required column names
        col_collision = crash_cols.get("collisionid")
        col_city = crash_cols.get("cityname")
        col_primary = crash_cols.get("primaryroad")
        col_secondary = crash_cols.get("secondaryroad")
        col_secdir = crash_cols.get("secondarydirection")
        col_secdist = crash_cols.get("secondarydistance")
        col_lat = crash_cols.get("latitude")
        col_lon = crash_cols.get("longitude")
        col_dt = crash_cols.get("crashdatetime")
        if col_collision is None or col_city is None:
            raise ValueError(f"Required crash columns missing in {year} data")
        # Filter for San Francisco crashes
        sf_crashes = crashes[
            crashes[col_city].astype(str).str.strip().str.upper().eq("SAN FRANCISCO")
        ].copy()
        sf_crashes[col_collision] = sf_crashes[col_collision].astype(str).str.strip()
        sf_collision_ids = set(sf_crashes[col_collision].dropna().tolist())
        
        print(f"  SF crashes: {len(sf_collision_ids):,}")
        # Process parties data in chunks
        chunk_iter = load_parties_data_chunked(year)
        
        non_driver_age_rows = []
        bicyclist_party_count = 0
        bike_collision_ids = set()
        sf_party_row_count = 0
        for chunk_num, chunk in enumerate(chunk_iter, 1):
            if chunk_num % 5 == 0:
                print(f"    Processing chunk {chunk_num}...")
                
            party_cols = {norm(c): c for c in chunk.columns}
            p_col_collision = party_cols.get("collisionid")
            p_col_partytype = party_cols.get("partytype")
            p_col_age = party_cols.get("statedage")
            if p_col_collision is None or p_col_partytype is None:
                raise ValueError(f"Required party columns missing in {year} data")
            # Filter for SF parties
            chunk[p_col_collision] = chunk[p_col_collision].astype(str).str.strip()
            sf_parties = chunk[chunk[p_col_collision].isin(sf_collision_ids)].copy()
            if sf_parties.empty:
                continue
            sf_party_row_count += len(sf_parties)
            party_type_upper = (
                sf_parties[p_col_partytype].astype(str).str.strip().str.upper()
            )
            # Collect non-driver ages
            non_driver_mask = ~party_type_upper.eq("DRIVER")
            if p_col_age is not None:
                ages = sf_parties.loc[non_driver_mask, [p_col_age]].copy()
                ages.columns = ["StatedAge"]
                non_driver_age_rows.append(ages)
            # Count bicyclists
            bike_mask = party_type_upper.str.contains("BICYCL", na=False)
            bicyclist_party_count += int(bike_mask.sum())
            if bike_mask.any():
                bike_collision_ids.update(
                    sf_parties.loc[bike_mask, p_col_collision]
                    .dropna()
                    .astype(str)
                    .str.strip()
                    .tolist()
                )
        # Store non-driver ages for this year
        if non_driver_age_rows:
            all_non_driver_ages.append(pd.concat(non_driver_age_rows, ignore_index=True))
        # Store yearly bicyclist counts
        yearly_bicyclist_counts.append(
            {
                "Year": year,
                "BicyclistPartyCount": bicyclist_party_count,
                "UniqueBikeCrashCount": len(bike_collision_ids),
                "SFCollisionCount": len(sf_collision_ids),
                "SFPartyRowCount": sf_party_row_count,
            }
        )
        
        print(f"  Bicyclist parties: {bicyclist_party_count:,}")
        print(f"  Unique bike crashes: {len(bike_collision_ids):,}")
        # Extract bike crash locations
        if bike_collision_ids:
            location_cols = [
                c
                for c in [
                    col_collision,
                    col_primary,
                    col_secondary,
                    col_secdir,
                    col_secdist,
                    col_lat,
                    col_lon,
                    col_dt,
                ]
                if c is not None
            ]
            bike_locations = sf_crashes[
                sf_crashes[col_collision].astype(str).str.strip().isin(bike_collision_ids)
            ][location_cols].copy()
            bike_locations["Year"] = year
            # Standardize column names
            rename_map = {
                col_collision: "CollisionId",
                col_primary: "PrimaryRoad",
                col_secondary: "SecondaryRoad",
                col_secdir: "SecondaryDirection",
                col_secdist: "SecondaryDistance",
                col_lat: "Latitude",
                col_lon: "Longitude",
                col_dt: "CrashDateTime",
            }
            bike_locations = bike_locations.rename(
                columns={k: v for k, v in rename_map.items() if k in bike_locations.columns}
            )
            # Build addresses
            bike_locations["Address"] = bike_locations.apply(build_address, axis=1)
            keep_cols = [
                c
                for c in [
                    "Year",
                    "CollisionId",
                    "Address",
                    "PrimaryRoad",
                    "SecondaryRoad",
                    "Latitude",
                    "Longitude",
                    "CrashDateTime",
                ]
                if c in bike_locations.columns
            ]
            bike_location_frames.append(
                bike_locations[keep_cols].drop_duplicates(subset=["CollisionId"])
            )
    # Process age frequency data
    if all_non_driver_ages:
        print("\n Processing age frequency data...")
        ages = pd.concat(all_non_driver_ages, ignore_index=True)
        ages["StatedAge"] = ages["StatedAge"].astype(str).str.strip()
        ages.loc[
            ages["StatedAge"].isin(["", "nan", "None"]), "StatedAge"
        ] = "(blank)"
        age_frequency = (
            ages.groupby("StatedAge", dropna=False)
            .size()
            .reset_index(name="Frequency")
        )
        # Clean up decimal formatting
        age_frequency["StatedAge"] = age_frequency["StatedAge"].apply(
            lambda v: str(v)[:-2]
            if str(v).endswith(".0") and str(v)[:-2].replace("-", "").isdigit()
            else str(v)
        )
        age_frequency = (
            age_frequency.groupby("StatedAge", as_index=False)["Frequency"]
            .sum()
            .reset_index(drop=True)
        )
        # Sort by numeric age where possible
        age_frequency["_age_num"] = pd.to_numeric(
            age_frequency["StatedAge"], errors="coerce"
        )
        age_frequency = (
            age_frequency.sort_values(
                ["_age_num", "StatedAge"], na_position="last"
            )
            .drop(columns=["_age_num"])
            .reset_index(drop=True)
        )
        nonblank_age_frequency = age_frequency[
            ~age_frequency["StatedAge"].isin(["(blank)", "", "nan", "None"])
        ].copy()
    else:
        age_frequency = pd.DataFrame(columns=["StatedAge", "Frequency"])
        nonblank_age_frequency = age_frequency.copy()
    # Process bicyclist counts
    bicyclist_counts = pd.DataFrame(yearly_bicyclist_counts).sort_values(
        "Year"
    ).reset_index(drop=True)
    # Process bike locations
    if bike_location_frames:
        bike_locations_final = pd.concat(bike_location_frames, ignore_index=True)
        bike_locations_final = bike_locations_final.sort_values(
            ["Year", "CollisionId"]
        ).reset_index(drop=True)
    else:
        bike_locations_final = pd.DataFrame(
            columns=[
                "Year",
                "CollisionId",
                "Address",
                "PrimaryRoad",
                "SecondaryRoad",
                "Latitude",
                "Longitude",
                "CrashDateTime",
            ]
        )
    print(" Analysis complete!")
    # --- Save pedestrian summary CSVs for visualization ---
    OUT_DIR = OUTPUT_DIR  # reuse previous variable if defined
    OUT_DIR.mkdir(exist_ok=True)

    # Create overall and year-level summary versions compatible with viz section
    overall_summary = age_freq_nonblank.copy()
    overall_path = OUT_DIR / "pedestrian_crash_summary_table.csv"
    overall_summary.to_csv(overall_path, index=False)

    yearly_summary_data = []
    for y in YEARS:
        subset = bike_counts[bike_counts["Year"] == y].copy()
        subset["source_year"] = y
        subset["age_group"] = "(not computed)"
        subset["sex"] = "(not computed)"
        subset["severity"] = "(not computed)"
        subset["crash_count"] = subset["BicyclistPartyCount"]
        yearly_summary_data.append(subset[["source_year", "age_group", "sex", "severity", "crash_count"]])
    summary_by_year = pd.concat(yearly_summary_data, ignore_index=True)
    yearly_path = OUT_DIR / "pedestrian_crash_summary_table_by_year.csv"
    summary_by_year.to_csv(yearly_path, index=False)

    print("\n Saved visualization summaries:")
    print(" -", overall_path)
    print(" -", yearly_path)

    return age_frequency, nonblank_age_frequency, bicyclist_counts, bike_locations_final
# Run the analysis
age_freq, age_freq_nonblank, bike_counts, bike_locations = run_analysis()

Starting SF Crash Analysis...

--- Processing 2021 ---
  SF crashes: 5,767
    Processing chunk 5...
  Bicyclist parties: 426
  Unique bike crashes: 418

--- Processing 2022 ---
  SF crashes: 5,769
  Bicyclist parties: 407
  Unique bike crashes: 400

--- Processing 2023 ---
  SF crashes: 5,967
    Processing chunk 5...
  Bicyclist parties: 405
  Unique bike crashes: 393

--- Processing 2024 ---
  SF crashes: 6,223
    Processing chunk 5...
  Bicyclist parties: 511
  Unique bike crashes: 497

--- Processing 2025 ---
  SF crashes: 6,094
    Processing chunk 5...
  Bicyclist parties: 998
  Unique bike crashes: 489

 Processing age frequency data...
 Analysis complete!

✅ Saved visualization summaries:
 - /workspaces/crash-data/analysis_outputs/pedestrian_crash_summary_table.csv
 - /workspaces/crash-data/analysis_outputs/pedestrian_crash_summary_table_by_year.csv


In [35]:
def save_results_and_display_summary(
    age_frequency: pd.DataFrame,
    nonblank_age_frequency: pd.DataFrame,
    bicyclist_counts: pd.DataFrame,
    bike_locations: pd.DataFrame
) -> None:
    """Save analysis results to CSV files and display summary."""
    
    # Define output file paths
    age_path = OUTPUT_DIR / "sf_non_driver_stated_age_frequency_2021_2025.csv"
    age_nonblank_path = OUTPUT_DIR / "sf_non_driver_stated_age_frequency_nonblank_2021_2025.csv"
    counts_path = OUTPUT_DIR / "sf_bicyclist_involved_counts_by_year_2021_2025.csv"
    bike_locations_path = OUTPUT_DIR / "sf_bicycle_crash_locations_2021_2025.csv"
    # Save to CSV files
    age_frequency.to_csv(age_path, index=False)
    nonblank_age_frequency.to_csv(age_nonblank_path, index=False)
    bicyclist_counts.to_csv(counts_path, index=False)
    bike_locations.to_csv(bike_locations_path, index=False)
    print("Output files saved:")
    print(f"  • {age_path}")
    print(f"  • {age_nonblank_path}")
    print(f"  • {counts_path}")
    print(f"  • {bike_locations_path}")
    
    print(f"\n Summary Statistics:")
    print(f"  • Total non-driver age records: {age_frequency['Frequency'].sum():,}")
    print(f"  • Non-blank age records: {nonblank_age_frequency['Frequency'].sum():,}")
    print(f"  • Total bicycle crash locations: {len(bike_locations):,}")
    
    print(f"\n Yearly Bicyclist Involvement:")
    display(bicyclist_counts)
    
    print(f"\n👥 Top 10 Non-Driver Ages (Non-blank):")
    if not nonblank_age_frequency.empty:
        display(nonblank_age_frequency.head(10))
    else:
        print("No non-blank age data available")
# Save results and display summary
save_results_and_display_summary(age_freq, age_freq_nonblank, bike_counts, bike_locations)

Output files saved:
  • /workspaces/crash-data/analysis_outputs/sf_non_driver_stated_age_frequency_2021_2025.csv
  • /workspaces/crash-data/analysis_outputs/sf_non_driver_stated_age_frequency_nonblank_2021_2025.csv
  • /workspaces/crash-data/analysis_outputs/sf_bicyclist_involved_counts_by_year_2021_2025.csv
  • /workspaces/crash-data/analysis_outputs/sf_bicycle_crash_locations_2021_2025.csv

 Summary Statistics:
  • Total non-driver age records: 15,426
  • Non-blank age records: 112
  • Total bicycle crash locations: 2,197

 Yearly Bicyclist Involvement:


,Year,BicyclistPartyCount,UniqueBikeCrashCount,SFCollisionCount,SFPartyRowCount
0,2021,426,418,5767,11963
1,2022,407,400,5769,11965
2,2023,405,393,5967,12450
3,2024,511,497,6223,12824
4,2025,998,489,6094,25019



👥 Top 10 Non-Driver Ages (Non-blank):


,StatedAge,Frequency
0,11,1
1,12,2
2,18,1
3,19,1
4,20,2
5,22,2
6,24,1
7,25,7
8,26,5
9,27,2


In [36]:
print("Sample Bicycle Crash Locations (First 10):")
if not bike_locations.empty:
    display(bike_locations.head(10))
else:
    print("No bicycle crash location data available")
print(f"\n Sample Age Frequency Data (First 10):")
if not age_freq.empty:
    display(age_freq.head(10))
else:
    print("No age frequency data available")
print("\nAnalysis Complete! All results have been saved to the 'analysis_outputs' directory.")

Sample Bicycle Crash Locations (First 10):


,Year,CollisionId,Address,PrimaryRoad,SecondaryRoad,Latitude,Longitude,CrashDateTime
0,2021,1459240,US-101 N/B (GGB EAST SIDEWALK) & GGB SOUTH TOWER,US-101 N/B (GGB EAST SIDEWALK),GGB SOUTH TOWER,37.811743,-122.477211,4/20/2021 11:41:00 AM
1,2021,1559388,US-101 N/B- GGB EAST SIDEWALK & NORTH TOWER OF...,US-101 N/B- GGB EAST SIDEWALK,NORTH TOWER OF GOLDEN GATE BRIDGE,37.822990,-122.478780,8/25/2021 9:34:00 AM
2,2021,1573712,EAST PARKING LOT OF GOLDEN GATE BRIGDE & LINCO...,EAST PARKING LOT OF GOLDEN GATE BRIGDE,LINCOLN BOULEVARD,37.807187,-122.474038,9/10/2021 9:00:00 AM
3,2021,2728135,GOETTINGEN ST & DWIGHT ST,GOETTINGEN ST,DWIGHT ST,NaN,NaN,3/8/2021 1:56:00 PM
4,2021,2733451,CLAY ST & DIVISADERO ST,CLAY ST,DIVISADERO ST,NaN,NaN,1/17/2021 9:35:00 AM
5,2021,2733452,18TH ST & LEXINGTON ST,18TH ST,LEXINGTON ST,NaN,NaN,1/16/2021 6:29:00 PM
6,2021,2733456,LOMBARD ST & THE EMBARCADERO,LOMBARD ST,THE EMBARCADERO,NaN,NaN,1/9/2021 2:17:00 PM
7,2021,2733461,BATTERY ST & BUSH ST,BATTERY ST,BUSH ST,NaN,NaN,1/19/2021 9:30:00 AM
8,2021,2733537,LAKE ST & 21ST ST,LAKE ST,21ST ST,NaN,NaN,1/6/2021 9:26:00 AM
9,2021,2733563,GEARY BL & 5TH AV,GEARY BL,5TH AV,NaN,NaN,2/1/2021 10:29:00 AM



 Sample Age Frequency Data (First 10):


,StatedAge,Frequency
0,11,1
1,12,2
2,18,1
3,19,1
4,20,2
5,22,2
6,24,1
7,25,7
8,26,5
9,27,2



Analysis Complete! All results have been saved to the 'analysis_outputs' directory.


In [37]:
# VISUALIZATIONS

!pip install matplotlib ipywidgets pandas numpy
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, Dropdown
plt.style.use("seaborn-v0_8-whitegrid")


[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [38]:
# Auto‑check and rebuild pedestrian summary files if missing
import pandas as pd
from pathlib import Path
# Reuse your existing OUTPUT_DIR variable from previous cells
summary_path = OUTPUT_DIR / "pedestrian_crash_summary_table.csv"
summary_by_year_path = OUTPUT_DIR / "pedestrian_crash_summary_table_by_year.csv"
if not summary_path.exists() or not summary_by_year_path.exists():
    print("Summary files missing – rebuilding them using main() ...")
    # Call the pedestrian analysis you already defined
    try:
        main()
    except Exception as e:
        raise RuntimeError("❌ Could not regenerate summary files") from e
else:
    print("Summary files already present")
# Now load them safely
summary_all = pd.read_csv(summary_path)
summary_by_year = pd.read_csv(summary_by_year_path)
print("Data loaded successfully:")
print(f" - {summary_path}")
print(f" - {summary_by_year_path}")

Summary files already present
Data loaded successfully:
 - /workspaces/crash-data/analysis_outputs/pedestrian_crash_summary_table.csv
 - /workspaces/crash-data/analysis_outputs/pedestrian_crash_summary_table_by_year.csv


In [39]:
# Load already-generated summary tables
OUT_DIR = OUTPUT_DIR  # Reuse from earlier notebook section
summary_by_year_path = OUT_DIR / "pedestrian_crash_summary_table_by_year.csv"
summary_path = OUT_DIR / "pedestrian_crash_summary_table.csv"
summary_all = pd.read_csv(summary_path)
summary_by_year = pd.read_csv(summary_by_year_path)
print("✅ Data loaded:")
print(f" - Overall summary rows: {len(summary_all):,}")
print(f" - Year-by-year summary rows: {len(summary_by_year):,}")

✅ Data loaded:
 - Overall summary rows: 52
 - Year-by-year summary rows: 5


In [40]:
# Configuration
AGE_ORDER = ["0-17", "18-24", "25-34", "35-44", "45-54", "55-64", "65+", "Unknown"]
SEX_ORDER = ["Female", "Male", "Non-binary", "Unknown"]
SEVERITY_ORDER = [
    "Fatal",
    "SuspectSerious",
    "SevereInactive",
    "SuspectMinor",
    "PossibleInjury",
    "ComplaintOfPainInactive",
    "OtherVisibleInactive",
    "Unknown",
]
def make_pivot_plot(df, title, chart_type="Count"):
    """Create either a count or percent stacked bar chart of crash severity mix."""
    if df.empty:
        fig, ax = plt.subplots(figsize=(10, 5))
        ax.axis("off")
        ax.text(0.5, 0.5, "⚠️ No data for this selection", ha="center", va="center", fontsize=14)
        plt.show()
        return
    df["age_group"] = pd.Categorical(df["age_group"], categories=AGE_ORDER, ordered=True)
    df["sex"] = pd.Categorical(df["sex"], categories=SEX_ORDER, ordered=True)
    pivot = (
        df.pivot_table(
            index=["age_group", "sex"],
            columns="severity",
            values="crash_count",
            aggfunc="sum",
            fill_value=0,
        )
        .reindex(columns=[c for c in SEVERITY_ORDER if c in df["severity"].unique()])
        .sort_index()
    )
    # Normalize to percent if selected
    if chart_type.lower() == "percent":
        pivot = pivot.div(pivot.sum(axis=1).replace(0, np.nan), axis=0) * 100
        ylabel = "Percent of Crashes"
    else:
        ylabel = "Crash Count"
    pivot.plot(kind="bar", stacked=True, figsize=(14, 6), width=0.85, alpha=0.9)
    plt.title(title, fontsize=14)
    plt.xlabel("Age Group | Sex")
    plt.ylabel(ylabel)
    plt.xticks(rotation=45, ha="right")
    plt.legend(title="Severity", bbox_to_anchor=(1.02, 1), loc="upper left")
    plt.tight_layout()
    plt.show()

In [ ]:
def get_summary_for_year(year_value: str) -> pd.DataFrame:
    """Filter summary by selected year or return all years."""
    if year_value == "All Years":
        return summary_all.copy()
    return summary_by_year.loc[
        summary_by_year["source_year"].astype(str) == year_value
    ].copy()
@interact(
    year=Dropdown(
        options=["All Years"] + [str(y) for y in sorted(summary_by_year["source_year"].astype(int).unique())],
        value="All Years",
        description="Year:",
    ),
    chart_type=Dropdown(
        options=["Count", "Percent"],
        value="Count",
        description="Chart Type:",
    ),
)
def interactive_sf_plot(year, chart_type):
    df = get_summary_for_year(year)
    make_pivot_plot(df, f"San Francisco Pedestrian Crashes ({year})", chart_type)